In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from loguru import logger
logger.remove()
logger.add(sys.stdout, level="INFO")

1

In [3]:
logger.remove()
logger.add(sys.stdout, level="DEBUG")

2

In [5]:
import sys
import os
sys.path.insert(0, '../')

from llmagent.privatestore import WebStore, WebConf
from llmagent.secret import AK_SK
from llmagent.llmapi import model_spec, QwenEmbedding, QwenLLM
qw_key = AK_SK("../llmagent/secret/keystore/qwen.keys")
os.environ["OPENAI_API_KEY"] = qw_key.ak
llm = QwenLLM(model_spec=model_spec.QwenPlus, temperature=0.6)
embedding = QwenEmbedding(api_key=qw_key.ak)
webstore = WebStore("llmtutorial", "../vectorstore/llmtutorial_qwen", emb_func=embedding, extra_config=WebConf(
    root_url="https://aitutor.liduos.com/",
    limit=10,
))

In [7]:
webstore.reload()

/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/

2024-10-25 09:26:35.931 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 0 to 16
2024-10-25 09:26:36.704 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 16 to 32
2024-10-25 09:26:37.485 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 32 to 48
2024-10-25 09:26:38.262 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 48 to 64
2024-10-25 09:26:38.943 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 64 to 80
2024-10-25 09:26:39.674 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 80 to 96
2024-10-25 09:26:40.390 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 96 to 112
2024-10-25 09:26:41.295 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 112 to 128
2024-10-25 09:26:42.228 | INFO     | llmagent.privatestore.base:__load__:52 - Adding documents 128 to 144
2024-10-25 09:26:42.932 | INFO     | llmagent.privatestore.b

In [8]:
webstore.collection_count()

1393

In [9]:
webstore._documents['https://aitutor.liduos.com/01-llm/01-2.html'].page_content

'作者：莫尔索 (https://liduos.com/about)     前言 (../)    关注《莫尔索随笔》 (https://liduos.com/wxqcode.png)    大语言模型概述    大语言模型概况 (01-1.html)    你好, ChatGPT (01-2.html)    OpenAI 文档解读 (01-3.html)    动手实现聊天机器人 (01-4.html)    基于 OpenAI API 搭建一个端到端问答系统 (01-5.html)    LLM 安全专题 (01-6.html)      LangChain入门    LangChain介绍 (../02-langchain/02-1.html)    LangChain模块学习 (../02-langchain/02-2.html)    LangChain之Chains模块 (../02-langchain/02-2-1.html)    LangChain之Agents模块 (../02-langchain/02-2-2.html)    LangChain之Callback模块 (../02-langchain/02-2-3.html)    Embedding嵌入 (../02-langchain/02-3.html)    动手实现文档问答机器人 (../02-langchain/02-4.html)      LlamaIndex 概述    LlamaIndex介绍 (../03-llamaIndex/03-1.html)    LlamaIndex索引 (../03-llamaIndex/03-2.html)    动手实现企业知识库 (../03-llamaIndex/03-3.html)      HuggingGPT 实现    HuggingFace 介绍 (../04-huggingface/04-1.html)    transformers 库基础组件 (../04-huggingface/04-2.html)    多模态任务设计 (../04-huggingface/04-3.html)    动手实现 HuggingGPT (../04-huggingface/04-4.html)      LLMOps 专题    L

## 构建Prompt

In [10]:
from llmagent.chain import QAWithContextChain, TemplateConf
prompt = QAWithContextChain(llm=llm, 
                               vectorstore=webstore.vectorstore(),
                               conf=TemplateConf(context_num=10, history_num=3))

In [14]:
import time
ans = prompt.stream("langchain提示词管理")
for token in ans:
    for c in token:
        time.sleep(0.1)
        print(c, end="")
    

2024-10-25 09:37:48.101 | DEBUG    | llmagent.llmapi.llm:stream:111 - input: [SystemMessage(content='你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：\nTitle    LLM 安全专题 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   LLM 安全专题  提示词 是指在训练或与大型语言模型（Claude，ChatGPT等）进行交互时，提供给模型的输入文本。通过给定特定的 提示词，可以引导模型生成特定主题或类型的文本。在自然语言处理（NLP）任务中，提示词充当了问题或输入的角色，而模型的输出是对这个问题的回答或完成的任务。\nsource: https://aitutor.liduos.com/01-llm/01-6.html\n\n提示词管理   提示模板 动态提示词=提示模板+变量，通过引入给提示词引入变量的方式，一方面保证了灵活性，一方面又能保证Prompt内容结构达到最佳   from langchain import PromptTemplate no_input_prompt = PromptTemplate(input_variables=[], template= "Tell me a joke." ) no_input_prompt.format() one_input_prompt = PromptTemplate(input_variables=[ "adjective" ], template=\nsource: https://aitutor.liduos.com/02-langchain/02-2.html\n\nModel I/O (https://python.langchain.com/docs/modules/model_io/)  这部分包括对大语言模型输入输出的管理，输入环节的提示

In [30]:
ans = prompt.stream("langchain回调管理器如何使用")
for token in ans:
    for c in token:
        time.sleep(0.03)
        print(c, end="")

2024-10-24 09:12:30.621 | DEBUG    | llmagent.llmapi.llm:_stream:38 - Prompt: 你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：
LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉问题，这对于日志记录、监视、流式处理和其他任务非常有用，现在也有专用的工具Helicone，Arize AI等产品可用，具体看 LLM应用生态初创公司说明 (../ref/company.md)  自定义回调对象  所有的回调对象都是基于这个基类来声明的  class  BaseCallbackHandler :  """Base callback handler that
source: https://aitutor.liduos.com/02-langchain/02-2-3.html

LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉问题，这对于日志记录、监视、流式处理和其他任务非常有用，现在也有专用的工具Helicone，Arize AI等产品可用，具体看 LLM应用生态初创公司说明 (../ref/company.md)  自定义回调对象  所有的回调对象都是基于这个基类来声明的  class  BaseCallbackHandler :  """Base callback handler that
source: https://aitutor.liduos.com/02-langchain/02-2-3.html

LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉

[WARNING][2024-10-24 09:12:30.622] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Speed-128K` will accept


在LangChain中，回调管理器是一个非常有用的模块，它允许开发者在大型语言模型（LLM）应用程序的各个阶段插入自定义回调函数，以实现日志记录、监视、流式处理等其他任务。为了使用LangChain的回调管理器，你可以遵循以下步骤：

1. 自定义回调对象：所有的回调对象都是基于`BaseCallbackHandler`这个基类来声明的。你可以根据自己的需求继承这个基类并实现特定的功能。2. 注册回调：在LangChain应用程序中，你需要将自定义的回调对象注册到回调管理器中。这样，当应用程序的特定阶段触发时，相应的回调函数就会被调用。3. 实现回调函数：在自定义的回调对象中，你需要实现具体的回调函数。这些函数将根据应用程序的阶段和事件进行触发，并执行相应的任务，如日志记录、数据监视等。4. 配置和使用：根据LangChain的文档和示例，配置回调管理器的使用。这可能涉及到在配置文件中指定回调对象的参数，或者在代码中直接实例化并配置回调对象。请注意，由于LangChain是一个不断发展和迭代的框架，具体的API和使用方式可能会有所变化。为了获得最准确和最新的信息，建议查阅LangChain的官方文档、参考示例代码或咨询相关领域的专业人士。此外，对于大型语言模型（LLM）的幻觉问题，回调管理器也可以用于日志记录和监视，以帮助识别和减少模型的幻觉。这也是使用LangChain回调管理器的一个重要应用场景。希望以上信息对你有所帮助！如果你还有其他问题，欢迎继续提问。

In [38]:
prompt.conf.history_num=2

In [35]:
prompt._get_history()

[HumanMessage(content='langchain回调管理器如何使用', additional_kwargs={}, response_metadata={}),
 AIMessage(content='在LangChain中，回调管理器是一个非常有用的模块，它允许开发者在大型语言模型（LLM）应用程序的各个阶段插入自定义回调函数，以实现日志记录、监视、流式处理等其他任务。为了使用LangChain的回调管理器，你可以遵循以下步骤：\n\n1. 自定义回调对象：所有的回调对象都是基于`BaseCallbackHandler`这个基类来声明的。你可以根据自己的需求继承这个基类并实现特定的功能。2. 注册回调：在LangChain应用程序中，你需要将自定义的回调对象注册到回调管理器中。这样，当应用程序的特定阶段触发时，相应的回调函数就会被调用。3. 实现回调函数：在自定义的回调对象中，你需要实现具体的回调函数。这些函数将根据应用程序的阶段和事件进行触发，并执行相应的任务，如日志记录、数据监视等。4. 配置和使用：根据LangChain的文档和示例，配置回调管理器的使用。这可能涉及到在配置文件中指定回调对象的参数，或者在代码中直接实例化并配置回调对象。请注意，由于LangChain是一个不断发展和迭代的框架，具体的API和使用方式可能会有所变化。为了获得最准确和最新的信息，建议查阅LangChain的官方文档、参考示例代码或咨询相关领域的专业人士。此外，对于大型语言模型（LLM）的幻觉问题，回调管理器也可以用于日志记录和监视，以帮助识别和减少模型的幻觉。这也是使用LangChain回调管理器的一个重要应用场景。希望以上信息对你有所帮助！如果你还有其他问题，欢迎继续提问。', additional_kwargs={}, response_metadata={})]

In [36]:
ans = prompt.stream("除了langchain，还有哪些ChatGPT框架")
for token in ans:
    for c in token:
        time.sleep(0.03)
        print(c, end="")

2024-10-24 09:13:24.433 | DEBUG    | llmagent.llmapi.llm:_stream:38 - Prompt: 你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：
the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 LangChain，介绍了如何基于 LangChain 框架开发具备实用功能、能力全面的应用程序，《LangChain Chat With Your Data》教程则在此基础上进一步介绍了如何使用 LangChain 架构结合个人私有数据开发个性化大模型应用；《Building Generative AI
source: https://aitutor.liduos.com/ref/ref.html

the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 LangChain，介绍了如何基于 LangChain 框架开发具备实用功能、能力全面的应用程序，《LangChain Chat With Your Data》教程则在此基础上进一步介绍了如何使用 LangChain 架构结合个人私有数据开发个性化大模型应用；《Building Generative AI
source: https://aitutor.liduos.com/ref/ref.html

the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 Lang

[WARNING][2024-10-24 09:13:24.434] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Speed-128K` will accept


除了LangChain，还有以下几个ChatGPT框架：

1. OpenAI的GPT-3框架：这是目前最流行的ChatGPT框架之一，由OpenAI开发和维护，提供各种基于AI的对话系统的能力，API可以进行即席试用和探索式迁移。开发人员可灵活地按需重建查询系统和下一代逻辑应用。GPT-3框架具有强大的自然语言处理能力，可实现高效的对话系统构建。2. Hugging Face的Transformers库：这是一个广泛使用的自然语言处理库，提供了许多预训练的语言模型，包括ChatGPT模型。Transformers库支持多种语言和任务，并且有丰富的文档和示例代码，方便开发者使用。Transformers库还提供了丰富的工具和功能，帮助开发者构建高效的对话系统。请注意，这些框架的使用方法各不相同。如果需要详细了解其特点和使用方式，请访问相关的官方网站获取最新的技术信息和支持文档。这些技术也在不断演进和更新中，因此建议查阅最新的官方文档或咨询专业人士以获取最新信息。

In [39]:
prompt.get_history_wo_role()

'langchain回调管理器如何使用\n除了langchain，还有哪些ChatGPT框架'